In [1]:
import json
import glob
import plotly.graph_objects as go

In [2]:
files = glob.glob('RandomSearch/**/trial*.json', recursive=True)

In [3]:
len(files)

600

In [4]:
trials = []

for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        file_parts = file.split('/')
        data['mode'] = file_parts[1]
        data['optimizer'] = file_parts[3]
        trials.append(data)        

In [5]:
trials.sort(key=lambda x: (x['metrics']['metrics']['val_loss']['observations'][0]['value'][0], -x['metrics']['metrics']['val_accuracy']['observations'][0]['value'][0]))

In [6]:
for trial in trials[:20]:
    print(trial['optimizer'], trial['mode'])

Adam FeatureScaling
Adam FeatureScaling
Adam FeatureScaling
Adam NonFeatureScaling
Adam FeatureScaling
Adam FeatureScaling
RMSprop FeatureScaling
Adam FeatureScaling
RMSprop FeatureScaling
Adam FeatureScaling
Adam FeatureScaling
RMSprop FeatureScaling
RMSprop FeatureScaling
Adam FeatureScaling
RMSprop FeatureScaling
Adam FeatureScaling
Adam FeatureScaling
Adam NonFeatureScaling
RMSprop FeatureScaling
Adam FeatureScaling


In [8]:
index = []
val_losses = []
val_accuracies = []
train_accuracies = []
trial_locations = []

for i,trial in enumerate(trials[:10]):
    val_loss = trial['metrics']['metrics']['val_loss']['observations'][0]['value'][0]
    val_accuracy = trial['metrics']['metrics']['val_accuracy']['observations'][0]['value'][0]
    train_accuracy = trial['metrics']['metrics']['accuracy']['observations'][0]['value'][0]
    trial_location = [trial['mode'], trial['optimizer'], trial['trial_id']]
    
    index.append(i+1)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    train_accuracies.append(train_accuracy)
    trial_locations.append(trial_location)

In [35]:
# Create a scatter plot
fig = go.Figure()

# Add a trace for validation loss
fig.add_trace(go.Scatter(
    x=index,
    y=val_losses,
    mode='lines+markers',
    name='Validation Loss',
    text=trial_locations,
    marker=dict(
        size=10,
        color='rgba(152, 0, 0, .8)',
        line=dict(
            width=2,
        )
    )
))

# Add a trace for validation accuracy
fig.add_trace(go.Scatter(
    x=index,
    y=val_accuracies,
    mode='lines+markers',
    name='Validation Accuracy',
    text=trial_locations,
    marker=dict(
        size=10,
        color = 'rgba(0, 152, 0, .8)',
        line=dict(
            width=2,
        )
    )
))

# Add a trace for training accuracy
fig.add_trace(go.Scatter(
    x=index,
    y=train_accuracies,
    mode='lines+markers',
    name='Training Accuracy',
    text=trial_locations,
    marker=dict(
        size=10,
        color='rgba(255, 182, 193, .9)',
        line=dict(
            width=2,
        )
    )
))

# Set the title and labels
fig.update_layout(
    title='Hyperparameter Tuning Results',
    xaxis_title='Index',
    yaxis_title='Accuracy',
    legend_title='Metrics',
)

# Show the plot
fig.show()

In [13]:
TF_ENABLE_ONEDNN_OPTS=0
from tensorflow.keras.models import load_model
import pandas as pd

In [12]:
best_model = load_model('RandomSearch/FeatureScaling/sparceCategorical/Adam/logs/trial_063/model.keras')

In [15]:
data = pd.read_csv('Dataset/combine.csv')
FEATURE_SCALED = True

X = data.drop(['Height cm (H)','Width cm (W)','Lower Angle (L)','Upper Angle (U)','Internal Angle (Right,2nd)','Leave Model'],axis=1)
y = data['Leave Model']

if FEATURE_SCALED:
    from sklearn.preprocessing import MinMaxScaler
    # Feature Scalling
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(x_scaled, columns=['Ratio (H/W)','Ratio(L/U)','Ratio(L/I)'])
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
val_loss, val_accuracy = best_model.evaluate(X_test, y_test)

# Print the validation loss and accuracy
print('Validation loss: ', val_loss)
print('Validation accuracy: ', val_accuracy)

1/1 [==============================] - 0s 255ms/step - loss: 0.0041 - accuracy: 1.0000
Validation loss:  0.004136896692216396
Validation accuracy:  1.0


In [34]:
TRIAL_NUMBER = 6
hyper_parms= trials[TRIAL_NUMBER]['hyperparameters']['values']
for key, value in hyper_parms.items():
     print(f'{key}: {value}')

epoches = trials[TRIAL_NUMBER]['metrics']['metrics']['loss']['observations'][0]['step']
optimizer = trials[TRIAL_NUMBER]['optimizer']
mode = trials[TRIAL_NUMBER]['mode']
print('Epoches:', epoches)
print('Optimizer:',optimizer)
print('Mode:', mode)

units: 320
num_layers: 5
activation: tanh
learning_rate: 0.001
Epoches: 4
Optimizer: RMSprop
Mode: FeatureScaling
